<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_train_person_cigarette.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train a model for sigarette detection

In [8]:
from ultralytics import YOLO
import torch
from pathlib import Path
import json
from datetime import datetime
import psutil
from tqdm.auto import tqdm

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class ModelCallback:
    def __init__(self, save_dir, epochs, patience=10, min_improvement=0.001):
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.best_map = 0
        self.patience = patience
        self.min_improvement = min_improvement
        self.epochs_without_improvement = 0
        self.training_history = []
        self.epochs_pbar = tqdm(total=epochs, desc="Training Progress", position=0)
        self.current_epoch = 0
        
    def should_stop_training(self, val_map):
        """
        Determine if training should stop based on validation mAP improvements
        """
        if val_map > (self.best_map + self.min_improvement):
            self.best_map = val_map
            self.epochs_without_improvement = 0
            return False
        else:
            self.epochs_without_improvement += 1
            if self.epochs_without_improvement >= self.patience:
                print(f"\nStopping training: No improvement in mAP for {self.patience} epochs")
                print(f"Best mAP achieved: {self.best_map:.4f}")
                return True
        return False

    def on_train_batch_end(self, trainer):
        # Skip batch progress updates
        pass

    def on_train_epoch_end(self, trainer):
        metrics = trainer.metrics
        epoch = trainer.epoch
        current_map = metrics.get('metrics/mAP50-95(B)', 0)

        # Run validation and get mAP
        val_results = trainer.model.val(verbose=False) 
        val_map = val_results.box.map
        val_map50 = val_results.box.map50

        # Save training history
        self.training_history.append({
            'epoch': epoch,
            'train_mAP': current_map,
            'val_mAP': val_map,
            'val_mAP50': val_map50,
            'metrics': metrics
        })

        with open(self.save_dir / 'training_history.json', 'w') as f:
            json.dump(self.training_history, f, indent=4)

        # Save model if it's the best so far
        if val_map > self.best_map:
            best_path = self.save_dir / f'best_model_map_{val_map:.4f}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': trainer.model.state_dict(),
                'val_map': val_map,
                'val_map50': val_map50,
                'train_map': current_map
            }, best_path)

        # Always save last model
        last_path = self.save_dir / 'last_model.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': trainer.model.state_dict(),
            'val_map': val_map,
            'val_map50': val_map50,
            'train_map': current_map
        }, last_path)

        self.epochs_pbar.update(1)
        self.epochs_pbar.set_postfix({
            'train_mAP': f'{current_map:.4f}',
            'val_mAP': f'{val_map:.4f}',
            'no_improve': self.epochs_without_improvement
        }, refresh=True)

        # Check if training should stop
        return self.should_stop_training(val_map)

    def on_train_end(self, trainer):
        self.epochs_pbar.close()

In [ ]:
def train_model(yaml_path, save_dir, epochs=100, imgsz=1024, batch=16, patience=10, min_improvement=0.001):
    model = YOLO('yolov10n.pt')
    callback = ModelCallback(save_dir, epochs, patience, min_improvement)

    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=0 if torch.cuda.is_available() else 'cpu',
        save=True,
        project=str(save_dir),
        name='train',
        exist_ok=True,
        val=True,
        verbose=False,
    )

    return model, callback.best_map

In [ ]:
if __name__ == "__main__":
    yaml_path = '/content/drive/MyDrive/Photo person/data.yaml'
    save_dir = Path('/content/drive/MyDrive/pth_person_detect')
    #timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    #run_dir = save_dir / f'run_{timestamp}'

    model, best_map = train_model(
    yaml_path= yaml_path,
    save_dir= save_dir,
    epochs=50,
    patience=10,
    min_improvement=0.001
)